In [3]:
!python -m venv venv

In [ ]:
!venv/Scripts/activate

In [ ]:
%pip install llama-index-embeddings-openai
%pip install llama-index-readers-file
%pip install python-dotenv

In [ ]:
%pip install -U qdrant_client
%pip install -U sentence-transformers

In [11]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["data/Almoravid+Rules+of+Play+-+LIVING+RULES+(1).pdf"]).load_data()

In [ ]:
#documents[0].text
for d in documents :
    print(d.text)

text = "\n".join([doc.text for doc in documents])
#TODO clear text

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")

In [ ]:
print(len(text))

In [1]:
documents

NameError: name 'documents' is not defined

In [10]:
from llama_index.core.node_parser import (
SentenceSplitter,
SemanticSplitterNodeParser)
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()
splitter = SemanticSplitterNodeParser(
    buffer_size=1,
    breakpoint_percentile_threshold=95,
    embed_model=embed_model
    #,sentence_splitter=SentenceSplitter(chunk_size=1024)
)

#split_text = splitter.get_nodes_from_documents(documents=documents, show_progress=True)

In [13]:
from llama_index.core import Document

base_splitter = SentenceSplitter(chunk_size=512)
#d = [Document(text=doc.text) for doc in documents]
#splitter.get_nodes_from_documents([Document(text=text, id="1", doc_id="1")], show_progress=True)
nodes = splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/36 [00:00<?, ?it/s]

Parsing nodes:   3%|▎         | 1/36 [00:01<00:51,  1.46s/it]

Parsing nodes:   6%|▌         | 2/36 [00:02<00:48,  1.44s/it]

Parsing nodes:   8%|▊         | 3/36 [00:04<00:47,  1.44s/it]

Parsing nodes:  11%|█         | 4/36 [00:05<00:36,  1.16s/it]

Parsing nodes:  14%|█▍        | 5/36 [00:06<00:37,  1.20s/it]

Parsing nodes:  17%|█▋        | 6/36 [00:07<00:35,  1.18s/it]

Parsing nodes:  19%|█▉        | 7/36 [00:07<00:28,  1.03it/s]

Parsing nodes:  22%|██▏       | 8/36 [00:10<00:39,  1.40s/it]

Parsing nodes:  25%|██▌       | 9/36 [00:11<00:33,  1.22s/it]

Parsing nodes:  28%|██▊       | 10/36 [00:11<00:28,  1.10s/it]

Parsing nodes:  31%|███       | 11/36 [00:13<00:32,  1.28s/it]

Parsing nodes:  33%|███▎      | 12/36 [00:14<00:30,  1.27s/it]

Parsing nodes:  36%|███▌      | 13/36 [00:15<00:27,  1.18s/it]

Parsing nodes:  39%|███▉      | 14/36 [00:16<00:23,  1.07s/it]

Parsing nodes:  42%|████▏     | 15/36 [00:17<00:23,  1.14s

In [20]:
[ n.get_content() for n in nodes]

['© 2021 GMT Games, LLC\nSummary of Almoravid Changes from Nevsky  ....................2\n1. Introduction ........................................................................3\nGeneral Course of Play ..........................................................3 \nComponents ...........................................................................3\nGame Board and Map ............................................................3\nTaifa Politics ..........................................................................5\nLords and Vassals ..................................................................6\nForces ....................................................................................8\nAssets .....................................................................................8\nOther Markers ........................................................................9\nCards ......................................................................................9\n2. ',
 'Set

In [27]:
from llama_index.core.node_parser import SentenceSplitter

# Initialize the SentenceSplitter
splitter = SentenceSplitter()

# Split the text into sentences
sentences = splitter.split_text(text)

print(len(sentences))


# Print the sentences
# for sentence in sentences:
#     print(sentence)

43


In [ ]:
#sentence transformers chunkings
from sentence_transformers import SentenceTransformer

#model = SentenceTransformer('sentence-transformers/sentence-t5-large')
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

#TODO try different models https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

vectors = [model.encode(sentence) for sentence in sentences]

In [31]:
len(vectors)

43

In [ ]:
#TODO rest of semantic chunking https://docs.llamaindex.ai/en/stable/examples/node_parsers/semantic_chunking/

In [21]:
#Qdrant
from qdrant_client import models, QdrantClient
qdrant = QdrantClient(path="./qdrant_data")

In [34]:
print("Current Qdrant collections: ")
for collection in qdrant.get_collections().collections:
    print(f"\t{collection}")

Current Qdrant collections: 
	name='demo_cosine'
	name='levycampaign_sentencesplitter_paraphraseMiniLML6v2'
	name='levycampaign_semanticsplitter_paraphraseMiniLML6v2'


[CollectionDescription(name='demo_cosine'),
 CollectionDescription(name='levycampaign_sentencesplitter_paraphraseMiniLML6v2'),
 CollectionDescription(name='levycampaign_semanticsplitter_paraphraseMiniLML6v2')]

In [33]:
len(vectors[0])

384

In [ ]:
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter, SemanticSplitterNodeParser
from llama_index.embeddings.openai import OpenAIEmbedding

# Assuming game_docs is a list of file paths
game_docs = ["data/Almoravid+Rules+of+Play+-+LIVING+RULES+(1).pdf"]

# Load documents
documents = SimpleDirectoryReader(input_files=game_docs).load_data()

# Convert loaded documents to Document objects
documents = [Document(text=doc.text) for doc in documents]

# Initialize the SentenceSplitter
base_splitter = SentenceSplitter(chunk_size=1024)

# Initialize the SemanticSplitter
embed_model = OpenAIEmbedding()
splitter = SemanticSplitterNodeParser(
    buffer_size=1,
    breakpoint_percentile_threshold=95,
    embed_model=embed_model    
)

# Split the documents using the SemanticSplitter
split_documents = splitter.get_nodes_from_documents(documents=documents, show_progress=True)

# Print the result
for doc in split_documents:
    print(doc)

In [ ]:
#TODO get rid of deprecation warning
qdrant.recreate_collection(
    #TODO different names for different embeddings
    collection_name="demo_cosine",
    vectors_config=models.VectorParams(
        size=384, # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

C:\Users\kasta\AppData\Local\Temp\ipykernel_23632\1308980876.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [ ]:
[x for x in enumerate(sentences)]

In [ ]:
vectors[0]

In [43]:
qdrant.upsert(
    collection_name="demo_cosine",
    points=[
        models.PointStruct(
            id=idx, 
            vector=vectors[idx].tolist(),
            # place where dict with the vector source to be added or id/reference for the source
            payload={"source": doc},
        )
        for idx, doc in enumerate(sentences)
    ],
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [2]:
searchedText = """Please explain to me rules of Battle in Almoravid"""

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

c:\Work\src\levycampaign\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
hits = qdrant.search(
    collection_name="demo_cosine",
    query_vector=model.encode(searchedText).tolist(),
    limit=5,
)

for hit in hits:
    print(hit)
    print("score:", hit.score, "*****TXT******", hit.payload["source"])

id=40 version=0 score=0.5186433699364841 payload={'source': '6.3.4 Plowing. At the end of the second 40 Days of Winter (box \n8), each Lord at a Siege (only) reduces his Carts and Mules each \nto ½ their number (rounded up).\n6.3.5 Arts of War. In the first Spring Levy (box 9), \ndraw Capabilities instead of Events (3.1).\n© 2021 GMT Games, LLC\nAlmoravid\n35\nKey Terms\n40 Days—Calendar box, a turn (1.3.2). \nActive—Acting on Campaign (4.0).\nAdjacent—Linked by a Way (1.3.1). \nAfrican Foot and Horse—Forces types (1.6). \nAllegiance—Friendly, Neutral, Enemy (1.3.1).\nApproach—March to enemy (4.3.4). \nArmor—Unit Protection roll (4.4.2). \nArray—Lords in Battle or Storm (4.4.1). \nArts of War—card, deck (1.9.1, 3.1). \nAsset—Coin, Loot, Provender, Transport (1.7). \nAttack—Engage in Battle or Storm as the Active side (4.4, 4.5.2). \nAutumn—a Season (2.2.1). \nAvoid Battle—Back off from Approach (4.3.4). \nBattle—Fight due to March (4.4). \nBattle mat—Array option (4.4.1). \nBesieged—Un

In [6]:
qdrant.close()